## Agent Based Simulation: Foraging & Cooperation

Let's build something a bit more complex. 

The simulation in this notebook is a simple model of early human resource use, predicated on the assumption that humans help each other. Let's give it the following key behaviour characteristics:
+ Resource Sharing: Agents share resources and energy with struggling neighbors
+ Knowledge Transfer: Successful cooperation increases cooperation levels
+ Trait Inheritance: Cooperative agents pass traits to offspring
+ Survival Advantage: Cooperation helps group survival

Before we go any further, what do you think will happen when it runs? When and how will cooperation emerge? When and under what conditions will it fail?:


In [ ]:
### Your thoughts:

.
.
.


## Step 1-2: World Setup

We have to define a world that the agents move around in, and we have to define the characterstics of the agents, and the actions that they can do.

Imagine if the top of your screen could fold down to meet the bottom, and the left side could be folded to meet the right: you would have a donut. That's the kind of world we're going to define for the agents (thus, if they wander off the right hand side of the world they reappear on the left, etc).

We create a 50x50 grid world with resource patches containing food, materials, and water.

In [ ]:
# preliminaries: gotta import the R libraries we'll use
library(NetLogoR)
library(ggplot2)
library(dplyr)

In [ ]:
# =============================================================================
# STEP 1: MODEL SETUP AND WORLD INITIALIZATION
# =============================================================================

# Define world dimensions
world_width <- 50
world_height <- 50


## Step 2: We'll place resources around the world

In [ ]:
# =============================================================================
# STEP 2: RESOURCE PATCHES
# =============================================================================

# Initialize resource patches on the landscape
initialize_resources <- function(n_resource_patches = 20, 
                                max_resource_value = 100) {
  # Create a data frame for resources
  resources <- data.frame(
    id = 1:n_resource_patches,
    xcor = runif(n_resource_patches, 0, world_width - 1),
    ycor = runif(n_resource_patches, 0, world_height - 1),
    resource_amount = runif(n_resource_patches, 20, max_resource_value),
    resource_type = sample(c("food", "materials", "water"), n_resource_patches, replace = TRUE),
    depleted = FALSE,
    stringsAsFactors = FALSE
  )
  return(resources)
}


## Step 3: Human Agents

Agents have attributes like energy, cooperation level, knowledge, and track helping behavior

In [ ]:
# =============================================================================
# STEP 3: HUMAN AGENTS
# =============================================================================

# Create human agents
create_humans <- function(n_humans = 15) {
  # Create a data frame for humans
  humans <- data.frame(
    id = 1:n_humans,
    xcor = runif(n_humans, 0, world_width - 1),
    ycor = runif(n_humans, 0, world_height - 1),
    energy = runif(n_humans, 50, 100),
    resources_carried = 0,
    cooperation_level = runif(n_humans, 0.3, 1.0),
    knowledge = runif(n_humans, 0.1, 0.8),
    help_given = 0,
    help_received = 0,
    age = 0,
    stringsAsFactors = FALSE
  )
  return(humans)
}

## Step 4: Movement

Humans move randomly but bias toward known resources based on their knowledge level

In [ ]:
# =============================================================================
# STEP 4: MOVEMENT AND FORAGING BEHAVIOR
# =============================================================================

# Human movement - random walk with some intelligence
move_humans <- function(humans, resources) {
  # Replaced NetLogoR's NLast() with standard R's nrow()
  for (i in 1:nrow(humans)) {
    if (humans$energy[i] > 0) {
      
      human_pos <- c(humans$xcor[i], humans$ycor[i])
      new_heading_rad <- runif(1, 0, 2 * pi) # Heading in radians for math
      step_size <- 1
      
      # If human has knowledge, bias movement toward known resources
      if (humans$knowledge[i] > 0.5 && runif(1) < humans$knowledge[i]) {
        available_resources <- resources[!resources$depleted, ]
        
        if (nrow(available_resources) > 0) {
          # Calculate distances to all available resources
          distances <- sqrt((available_resources$xcor - human_pos[1])^2 + 
                                (available_resources$ycor - human_pos[2])^2)
          nearest_resource_idx <- which.min(distances)
          
          # Move toward nearest resource
          target_pos <- c(available_resources$xcor[nearest_resource_idx], 
                          available_resources$ycor[nearest_resource_idx])
          
          dx <- target_pos[1] - human_pos[1]
          dy <- target_pos[2] - human_pos[2]
          new_heading_rad <- atan2(dy, dx)
        }
      }
      
      # Move the human manually instead of using NetLogoR::fd()
      humans$xcor[i] <- humans$xcor[i] + step_size * cos(new_heading_rad)
      humans$ycor[i] <- humans$ycor[i] + step_size * sin(new_heading_rad)
      
      # Enforce world boundaries (clamping)
      humans$xcor[i] <- max(0, min(world_width - 1, humans$xcor[i]))
      humans$ycor[i] <- max(0, min(world_height - 1, humans$ycor[i]))
      
      # Consume energy for movement
      humans$energy[i] <- humans$energy[i] - 1
    }
  }
  return(humans)
}

## Step 5: Resource Gathering

Humans gather resources within a radius, with efficiency based on their knowledge
Resources can be depleted over time

In [ ]:
# =============================================================================
# STEP 5: RESOURCE GATHERING (operates on data.frames)
# =============================================================================

# Gather resources from patches
gather_resources <- function(humans, resources, gathering_radius = 2) {
  for (i in 1:nrow(humans)) {
    if (humans$energy[i] > 0) {
      human_pos <- c(humans$xcor[i], humans$ycor[i])
      distances <- sqrt((resources$xcor - human_pos[1])^2 + (resources$ycor - human_pos[2])^2)
      nearby_resources <- which(distances <= gathering_radius & !resources$depleted)
      
      if (length(nearby_resources) > 0) {
        target_resource_idx <- nearby_resources[which.min(distances[nearby_resources])]
        gathering_efficiency <- 0.3 + (humans$knowledge[i] * 0.4)
        amount_gathered <- min(resources$resource_amount[target_resource_idx], gathering_efficiency * 20)
        
        resources$resource_amount[target_resource_idx] <- resources$resource_amount[target_resource_idx] - amount_gathered
        humans$resources_carried[i] <- humans$resources_carried[i] + amount_gathered
        
        # --- CHANGE HERE: You can experiment with how much energy humans get by fiddling here ---
        # I originally had 0.2, which was too low; everyone died quickly. 0.6 is more generous: but what does that mean?
        humans$energy[i] <- humans$energy[i] + (amount_gathered * 0.6)
        
        if (resources$resource_amount[target_resource_idx] <= 0) {
          resources$depleted[target_resource_idx] <- TRUE
        }
        humans$knowledge[i] <- min(1.0, humans$knowledge[i] + 0.01)
      }
    }
  }
  return(list(humans = humans, resources = resources))
}

## Step 6: Cooperation (Core Feature)

- Humans help nearby agents who are struggling (low energy/resources)
- Help is given based on individual cooperation levels
- Cooperative behavior increases cooperation tendencies over time

In [ ]:
# =============================================================================
# STEP 6: COOPERATION AND HELPING BEHAVIOR (operates on data.frames)
# =============================================================================
# Humans help each other based on cooperation levels
cooperation_behavior <- function(humans, help_radius = 5) {
  for (i in 1:nrow(humans)) {
    if (humans$energy[i] > 20 && humans$resources_carried[i] > 10) {
      human_pos <- c(humans$xcor[i], humans$ycor[i])
      
      distances <- sqrt((humans$xcor - human_pos[1])^2 + (humans$ycor - human_pos[2])^2)
      
      potential_recipients <- which(distances <= help_radius & distances > 0 & (humans$energy < 30 | humans$resources_carried < 5))
      
      if (length(potential_recipients) > 0 && runif(1) < humans$cooperation_level[i]) {
        neediness_score <- (30 - humans$energy[potential_recipients]) + (10 - humans$resources_carried[potential_recipients])
        most_needy_idx <- potential_recipients[which.max(neediness_score)]
        
        help_amount_resources <- min(5, humans$resources_carried[i] * 0.2)
        help_amount_energy <- min(10, humans$energy[i] * 0.1)
        
        humans$resources_carried[i] <- humans$resources_carried[i] - help_amount_resources
        humans$energy[i] <- humans$energy[i] - help_amount_energy
        
        humans$resources_carried[most_needy_idx] <- humans$resources_carried[most_needy_idx] + help_amount_resources
        humans$energy[most_needy_idx] <- humans$energy[most_needy_idx] + help_amount_energy
        
        humans$help_given[i] <- humans$help_given[i] + 1
        humans$help_received[most_needy_idx] <- humans$help_received[most_needy_idx] + 1
        
        humans$cooperation_level[i] <- min(1.0, humans$cooperation_level[i] + 0.005)
        humans$cooperation_level[most_needy_idx] <- min(1.0, humans$cooperation_level[most_needy_idx] + 0.005)
      }
    }
  }
  return(humans)
}

## Step 7: Population Dynamics

- Agents age and can die from low energy or old age
- Successful, cooperative agents can reproduce, passing on traits

In [ ]:
# =============================================================================
# STEP 7: SURVIVAL AND REPRODUCTION 
# =============================================================================
# Handle survival, aging, and simple reproduction
update_population <- function(humans, reproduction_threshold = 25) {
  if (nrow(humans) == 0) return(humans)
  
  humans$age <- humans$age + 1
  survivor_indices <- which(humans$energy > 0 & humans$age < 200)
  humans <- humans[survivor_indices, ]
  
  if (nrow(humans) == 0) return(humans)
  
  # --- CHANGES HERE: Lowered the bar for reproduction ---
  # Old conditions were too strict (energy > 80, resources > 20, coop > 0.6)
  successful_humans <- which(
    humans$energy > 65 & 
    humans$resources_carried > 15 & 
    humans$cooperation_level > 0.5 
  )
  
  # --- CHANGE HERE: Increased the probability of reproduction ---
  # I intially had probability was 0.1, making reproduction a rare lottery. You might change this value; what does that map to, historically?
  reproduction_prob <- 0.3 
  
  if (length(successful_humans) > 0 && nrow(humans) < reproduction_threshold && runif(1) < reproduction_prob) {
    parent_idx <- sample(successful_humans, 1)
    parent <- humans[parent_idx, ]
    
    new_human <- data.frame(
      id = max(humans$id) + 1,
      xcor = max(0, min(world_width - 1, parent$xcor + runif(1, -3, 3))),
      ycor = max(0, min(world_height - 1, parent$ycor + runif(1, -3, 3))),
      energy = 60,
      resources_carried = 0,
      cooperation_level = max(0.1, min(1.0, parent$cooperation_level + runif(1, -0.1, 0.1))),
      knowledge = parent$knowledge * 0.5,
      help_given = 0,
      help_received = 0,
      age = 0,
      stringsAsFactors = FALSE
    )
    humans <- rbind(humans, new_human)
  }
  
  return(humans)
}

## Step 8-9: Simulation & Analysis

- Runs the complete simulation and tracks key metrics
- Visualizes population, cooperation, and knowledge over time

In [ ]:
# =============================================================================
# STEP 8: MAIN SIMULATION FUNCTION 
# =============================================================================

run_simulation <- function(n_steps = 200, n_humans = 15, n_resources = 20) {
  resources <- initialize_resources(n_resources)
  humans <- create_humans(n_humans)
  results <- list()
  
  for (step in 1:n_steps) {
    if(nrow(humans) == 0) {
      cat("Population went extinct. Stopping simulation.\n")
      break
    }
    
    humans <- move_humans(humans, resources)
    
    gather_result <- gather_resources(humans, resources)
    humans <- gather_result$humans
    resources <- gather_result$resources
    
    humans <- cooperation_behavior(humans)
    humans <- update_population(humans)
    
    if (step %% 10 == 0) {
      results[[length(results) + 1]] <- data.frame(
        step = step,
        n_humans = nrow(humans),
        avg_energy = mean(humans$energy),
        avg_resources = mean(humans$resources_carried),
        avg_cooperation = mean(humans$cooperation_level),
        total_help_given = sum(humans$help_given),
        avg_knowledge = mean(humans$knowledge),
        resources_remaining = sum(resources$resource_amount[!resources$depleted])
      )
    }
    
    if (step %% 50 == 0) {
      cat("Step", step, "- Population:", nrow(humans), "- Avg Cooperation:", round(mean(humans$cooperation_level), 3), "\n")
    }
  }
  return(list(results = results, final_humans = humans, final_resources = resources))
}

In [ ]:
# =============================================================================
# STEP 9: ANALYSIS AND VISUALIZATION 
# =============================================================================

analyze_results <- function(simulation_output) {
  if (length(simulation_output$results) == 0) {
    cat("No results to analyze.\n")
    return(NULL)
  }
  
  # Use dplyr::bind_rows for safely combining the list of data frames
  results_df <- dplyr::bind_rows(simulation_output$results)
  
  p1 <- ggplot(results_df, aes(x = step)) +
    geom_line(aes(y = n_humans, color = "Population")) +
    geom_line(aes(y = avg_energy/5, color = "Avg Energy/5")) +
    geom_line(aes(y = avg_cooperation*20, color = "Cooperation*20")) +
    labs(title = "Population Dynamics and Cooperation Over Time", x = "Simulation Step", y = "Value") +
    theme_minimal() +
    scale_color_manual(name = "Metric", values = c("Population" = "blue", "Avg Energy/5" = "red", "Cooperation*20" = "green"))

  p2 <- ggplot(results_df, aes(x = step)) +
    geom_line(aes(y = total_help_given, color = "Total Help Given")) +
    geom_line(aes(y = avg_knowledge*100, color = "Avg Knowledge*100")) +
    labs(title = "Cooperation and Knowledge Development", x = "Simulation Step", y = "Value") +
    theme_minimal() +
    scale_color_manual(name = "Metric", values = c("Total Help Given" = "purple", "Avg Knowledge*100" = "orange"))
  
  return(list(data = results_df, plot1 = p1, plot2 = p2))
}

## Run!
All of the code so far has been about defining all of the different pieces of lego brick that go together towards making a simulation. Now, it's time to run the thing.

In [ ]:
# =============================================================================
# STEP 9: ANALYSIS AND VISUALIZATION
# =============================================================================

cat("Starting Early Human Resource Use Simulation...\n")
simulation_results <- run_simulation(n_steps = 200, n_humans = 15, n_resources = 60)

analysis <- analyze_results(simulation_results)

if (!is.null(analysis)) {
  print(analysis$plot1)
  print(analysis$plot2)
  
  final_data <- tail(analysis$data, 1)
  cat("\n=== SIMULATION SUMMARY ===\n")
  cat("Final Population:", final_data$n_humans, "\n")
  cat("Average Cooperation Level:", round(final_data$avg_cooperation, 3), "\n")
  cat("Total Help Events:", final_data$total_help_given, "\n")
  cat("Average Knowledge:", round(final_data$avg_knowledge, 3), "\n")
  cat("Resources Remaining:", round(final_data$resources_remaining, 2), "\n")
}

## Ok, let's experiment

We're going to re-arrange the existing code into a single function, so that we can set up experiments and try to learn under what conditions a sustainable human foraging society might emerge.

In [ ]:
# This is a modified version of the simulation function, optimized for sweeps.
# It takes more parameters and returns only the final summary data.
run_single_simulation <- function(n_steps = 200, 
                                  n_humans = 15, 
                                  n_resources = 25,
                                  initial_cooperation_range = c(0.3, 1.0),
                                  initial_knowledge_range = c(0.1, 0.8),
                                  reproduction_threshold = 25) {
  
  # --- Setup functions (create_humans now uses the new parameters) ---
  initialize_resources_headless <- function() {
    data.frame(id = 1:n_resources, xcor = runif(n_resources, 0, 50), ycor = runif(n_resources, 0, 50),
               resource_amount = runif(n_resources, 20, 100),
               resource_type = sample(c("food", "water"), n_resources, replace = TRUE),
               depleted = FALSE, stringsAsFactors = FALSE)
  }
  create_humans_headless <- function() {
    data.frame(id = 1:n_humans, xcor = runif(n_humans, 0, 50), ycor = runif(n_humans, 0, 50),
               energy = runif(n_humans, 50, 100), resources_carried = 0,
               cooperation_level = runif(n_humans, initial_cooperation_range[1], initial_cooperation_range[2]),
               knowledge = runif(n_humans, initial_knowledge_range[1], initial_knowledge_range[2]),
               help_given = 0, help_received = 0, age = 0, stringsAsFactors = FALSE)
  }
  
  # --- Initialize agents ---
  resources <- initialize_resources_headless()
  humans <- create_humans_headless()
  
  # --- Core logic functions (move, gather, cooperate, update) ---
  # These are copied directly from the previous refactored code, no changes needed.
  # For brevity, we assume they are defined in the environment. Or, for a fully
  # self-contained function, you would paste them all here.
  # Let's assume they are available for now.
  
  # --- Simulation Loop ---
  for (step in 1:n_steps) {
    if(nrow(humans) == 0) break # Stop if population is extinct
    
    humans <- move_humans(humans, resources)
    gather_result <- gather_resources(humans, resources)
    humans <- gather_result$humans
    resources <- gather_result$resources
    humans <- cooperation_behavior(humans)
    # Pass the reproduction_threshold to the update function
    humans <- update_population(humans, reproduction_threshold) 
  }
  
  # --- Return Final Summary Statistics ---
  if(nrow(humans) > 0) {
    final_summary <- data.frame(
      final_population = nrow(humans),
      avg_energy = mean(humans$energy),
      avg_resources_carried = mean(humans$resources_carried),
      avg_cooperation = mean(humans$cooperation_level),
      total_help_given = sum(humans$help_given),
      avg_knowledge = mean(humans$knowledge),
      extinction = FALSE
    )
  } else {
    final_summary <- data.frame(
      final_population = 0, avg_energy = NA, avg_resources_carried = NA,
      avg_cooperation = NA, total_help_given = NA, avg_knowledge = NA,
      extinction = TRUE
    )
  }
  
  return(final_summary)
}

# We also need to slightly modify update_population to accept the threshold
update_population <- function(humans, reproduction_threshold = 25) {
  if (nrow(humans) == 0) return(humans)
  humans$age <- humans$age + 1
  survivor_indices <- which(humans$energy > 0 & humans$age < 200)
  humans <- humans[survivor_indices, ]
  if (nrow(humans) == 0) return(humans)
  successful_humans <- which(humans$energy > 80 & humans$resources_carried > 20 & humans$cooperation_level > 0.6)
  
  # Use the new parameter here
  if (length(successful_humans) > 0 && nrow(humans) < reproduction_threshold && runif(1) < 0.1) {
    parent_idx <- sample(successful_humans, 1)
    parent <- humans[parent_idx, ]
    new_human <- data.frame(id = max(humans$id) + 1, xcor = max(0, min(49, parent$xcor + runif(1, -3, 3))),
                           ycor = max(0, min(49, parent$ycor + runif(1, -3, 3))), energy = 60, resources_carried = 0,
                           cooperation_level = max(0.1, min(1.0, parent$cooperation_level + runif(1, -0.1, 0.1))),
                           knowledge = parent$knowledge * 0.5, help_given = 0, help_received = 0, age = 0)
    humans <- rbind(humans, new_human)
  }
  return(humans)
}

In [ ]:
# install.packages("progress") # A nice library for progress bars
library(progress)

# Function to sweep the parameter space of the model
sweep_behavior_space <- function(param_grid, n_runs) {
  
  # Total number of simulations to run
  total_sims <- nrow(param_grid) * n_runs
  
  # Setup a progress bar
  pb <- progress_bar$new(
    format = "  Sweeping [:bar] :percent in :elapsed. ETA: :eta",
    total = total_sims, clear = FALSE, width = 60)
  
  # A list to store the results from all runs
  all_results <- list()
  
  # Loop through each parameter combination
  for (i in 1:nrow(param_grid)) {
    params <- param_grid[i, ]
    
    # Run the simulation n_runs times for this parameter set
    for (run in 1:n_runs) {
      pb$tick() # Update the progress bar
      
      # Run one simulation with the current parameter set
      run_summary <- run_single_simulation(
        initial_cooperation_range = c(params$min_coop, params$max_coop),
        initial_knowledge_range = c(params$min_know, params$max_know),
        reproduction_threshold = params$repro_thresh
      )
      
      # Combine the input parameters with the output summary
      full_summary <- cbind(params, run_id = run, run_summary)
      
      # Add to our list of results
      all_results[[length(all_results) + 1]] <- full_summary
    }
  }
  
  # Combine all results into a single, tidy data frame
  return(dplyr::bind_rows(all_results))
}

In [ ]:
# Function to create a heatmap of the results
plot_sweep_heatmap <- function(sweep_data, x_var, y_var, metric) {
  
  # Summarize the data by averaging the metric across all runs for each parameter combo
  summary_data <- sweep_data %>%
    group_by(across(all_of(c(x_var, y_var)))) %>%
    summarise(mean_metric = mean(.data[[metric]], na.rm = TRUE), .groups = "drop")
  
  ggplot(summary_data, aes(x = .data[[x_var]], y = .data[[y_var]], fill = mean_metric)) +
    geom_tile(color = "white") +
    scale_fill_viridis_c(option = "plasma") + # A nice color scale
    labs(
      title = paste("Behavior Space of", metric),
      subtitle = paste("Averaged over", max(sweep_data$run_id), "runs per setting"),
      x = x_var,
      y = y_var,
      fill = paste("Mean", metric)
    ) +
    theme_minimal()
}

In [ ]:
# 1. Define the parameter grid for the sweep
# We'll test how the minimum starting cooperation and knowledge affect the outcomes.
parameter_grid <- expand.grid(
  min_coop = c(0.1, 0.4, 0.7),
  max_coop = 1.0, # Keep max constant
  min_know = c(0.0, 0.2, 0.4),
  max_know = 0.8, # Keep max constant
  repro_thresh = 25 # Keep threshold constant
)

# 2. Run the sweep
# Let's do 10 runs for each of the 3x3=9 parameter settings. Total = 90 simulations.
# This may take a minute or two.
sweep_results <- sweep_behavior_space(param_grid = parameter_grid, n_runs = 10)

# 3. Visualize the results as heatmaps

# How does initial cooperation and knowledge affect the final population?
p_pop <- plot_sweep_heatmap(sweep_results, "min_coop", "min_know", "final_population")

# How do they affect the final average cooperation level?
p_coop <- plot_sweep_heatmap(sweep_results, "min_coop", "min_know", "avg_cooperation")

# How do they affect the probability of extinction?
p_ext <- plot_sweep_heatmap(sweep_results, "min_coop", "min_know", "extinction")


# Print the plots
print(p_pop)
print(p_coop)
print(p_ext)

When you run your experiment, do you find a sweet spot where where both initial knowledge and cooperation must be above a certain threshold for the population to thrive? And if you do, what does that imply?

## Extending the model

Did you notice an assumption about resources in the model? Once a resource is exploited, it's gone. How about we make something a bit more realistic?

- Replenishment: Depleted patches slowly regain their resources over time. This creates static "hotspots" on the map that agents can learn and return to.

- Respawning: Depleted patches are removed, and new ones appear randomly elsewhere in the world. This keeps the total number of patches constant but makes the environment more dynamic and unpredictable.

In [ ]:
# =============================================================================
# NEW STEP: RESOURCE DYNAMICS
# =============================================================================

# Replenish depleted resource patches over time.
#
# This function models resources slowly growing back in the same location.
# @param resources The current resources data frame.
# @param replenish_prob The probability that any single depleted patch will start regrowing in a given tick.
# @param min_val The minimum resource value for a newly replenished patch.
# @param max_val The maximum resource value for a newly replenished patch.
# @return The updated resources data frame.
replenish_resources <- function(resources, replenish_prob = 0.05, min_val = 20, max_val = 100) {
  # Find which patches are currently depleted
  depleted_indices <- which(resources$depleted)
  
  if (length(depleted_indices) > 0) {
    # Each depleted patch has a chance to start replenishing
    replenish_roll <- runif(length(depleted_indices))
    indices_to_revive <- depleted_indices[replenish_roll < replenish_prob]
    
    if (length(indices_to_revive) > 0) {
      # For the revived patches, reset their status and resource amount
      resources$depleted[indices_to_revive] <- FALSE
      resources$resource_amount[indices_to_revive] <- runif(length(indices_to_revive), min_val, max_val)
    }
  }
  return(resources)
}


# Respawn depleted resource patches at new random locations.
#
# This function models a dynamic world where old resources disappear and new ones emerge elsewhere, keeping the total number of patches constant.
# @param resources The current resources data frame.
# @param min_val The minimum resource value for a newly respawned patch.
# @param max_val The maximum resource value for a newly respawned patch.
# @return The updated resources data frame.
respawn_resources <- function(resources, min_val = 20, max_val = 100) {
  # Find which patches are currently depleted
  depleted_indices <- which(resources$depleted)
  n_to_respawn <- length(depleted_indices)
  
  if (n_to_respawn > 0) {
    # Step 1: Remove the depleted patches from the data frame
    resources <- resources[-depleted_indices, ]
    
    # Step 2: Create new patches to replace them
    new_patches <- data.frame(
      # Give them new unique IDs
      id = (max(resources$id, 0) + 1):(max(resources$id, 0) + n_to_respawn),
      # Assign new random locations and values
      xcor = runif(n_to_respawn, 0, world_width - 1),
      ycor = runif(n_to_respawn, 0, world_height - 1),
      resource_amount = runif(n_to_respawn, min_val, max_val),
      resource_type = sample(c("food", "materials", "water"), n_to_respawn, replace = TRUE),
      depleted = FALSE,
      stringsAsFactors = FALSE
    )
    
    # Step 3: Add the new patches to the main data frame
    resources <- rbind(resources, new_patches)
  }
  return(resources)
}

In [ ]:
# The main simulation function, now with a choice of resource dynamics.
run_simulation <- function(n_steps = 200, 
                           n_humans = 15, 
                           n_resources = 25, 
                           resource_dynamics = "replenish") { 
  
  # Initialize world
  resources <- initialize_resources(n_resources)
  humans <- create_humans(n_humans)
  results <- list()
  
  # Run simulation steps
  for (step in 1:n_steps) {
    # Stop if population goes extinct
    if(nrow(humans) == 0) {
      cat("Population went extinct. Stopping simulation.\n")
      break
    }
    
    # 1. Agent Actions
    humans <- move_humans(humans, resources)
    gather_result <- gather_resources(humans, resources)
    humans <- gather_result$humans
    resources <- gather_result$resources # Make sure to update resources here
    humans <- cooperation_behavior(humans)
    humans <- update_population(humans)
    
    # 2. NEW: World Update Step
    # Based on the parameter, update the resources in the world
    if (resource_dynamics == "replenish") {
      resources <- replenish_resources(resources)
    } else if (resource_dynamics == "respawn") {
      resources <- respawn_resources(resources)
    }
    # If "static", we do nothing.
    
    # 3. Data Logging
    if (step %% 10 == 0) {
      # (The logging code remains the same as before)
      results[[length(results) + 1]] <- data.frame(
        step = step, n_humans = nrow(humans), avg_energy = mean(humans$energy),
        avg_resources = mean(humans$resources_carried), avg_cooperation = mean(humans$cooperation_level),
        total_help_given = sum(humans$help_given), avg_knowledge = mean(humans$knowledge),
        resources_remaining = sum(resources$resource_amount[!resources$depleted])
      )
    }
    
    if (step %% 50 == 0) {
      cat("Step", step, "- Population:", nrow(humans), "- Avg Cooperation:", round(mean(humans$cooperation_level), 3), "\n")
    }
  }
  
  return(list(results = results, final_humans = humans, final_resources = resources))
}

In [ ]:
# --- Example 1: Run with REPLENISHING resources ---
cat("Starting simulation with REPLENISHING resources...\n")
sim_replenish <- run_simulation(n_steps = 1000, resource_dynamics = "replenish")
analysis_replenish <- analyze_results(sim_replenish)

print(analysis_replenish$plot1)


# --- Example 2: Run with RESPAWNING resources ---
cat("\nStarting simulation with RESPAWNING resources...\n")
sim_respawn <- run_simulation(n_steps = 1000, resource_dynamics = "respawn")
analysis_respawn <- analyze_results(sim_respawn)

print(analysis_respawn$plot1)


In [ ]:
# --- Example 3: Run with Static resources ---
cat("\nStarting simulation with STATIC resources...\n")
sim_static <- run_simulation(n_steps = 1000,)
analysis_static <- analyze_results(sim_static)

print(analysis_static$plot1)